# Исследование уровня потребительской лояльности телекоммуникационой компании

***Заказчик этого исследования*** — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.    

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».   

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.    


# Вопросы исследования
- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

# Задачи

- Определить какие поля из таблиц понадобятся в работе
- Подключиться к базе
- Подготовить SQL-запрос и выгрузить необходимые данные
- Импортировать в Tableau получившуюся выгрузку
- Рассчитать NPS по формуле: % «сторонников» - % «критиков»
- Подготовить графики и дашборды, необходимые для ответа на вопросы исследования
- Сделать выводы
- Подготовить и опубликовать презентацию

### Подключение к базе

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

### Создание датасета 

In [8]:
query = """
SELECT u.user_id,
u.lt_day,
(CASE 
WHEN u.lt_day <= 365 THEN 'да'
ELSE 'нет'
END) as is_new,
u.age,
(CASE 
WHEN u.gender_segment = 1 THEN 'женщина'
WHEN u.gender_segment = 0 THEN 'мужчина'
ELSE 'unknown'
END) as gender_segment,
u.os_name,
u.cpe_type_name,
l.country,
l.city,
asg.title as age_segment,
ts.title as traffic_segment,
ls.title as lifetime_segment,
u.nps_score,
(CASE
WHEN u.nps_score <= 6 THEN 'критик'
WHEN u.nps_score > 6 AND u.nps_score < 9 THEN 'нейтрал'
WHEN u.nps_score >= 9 THEN 'cторонник'
END) as nps_group
FROM user as u
LEFT OUTER JOIN location AS l ON u.location_id = l.location_id
LEFT OUTER JOIN age_segment AS asg ON u.age_gr_id = asg.age_gr_id
LEFT OUTER JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id;
"""

In [9]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,нет,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,cторонник
1,A001WF,2344,нет,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,cторонник
2,A003Q7,467,нет,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,cторонник
3,A004TB,4190,нет,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,cторонник
4,A004XT,1163,нет,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,cторонник
5,A005O0,5501,нет,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,критик
6,A0061R,1236,нет,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,cторонник
7,A009KS,313,да,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,cторонник
8,A00AES,3238,нет,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,cторонник
9,A00F70,4479,нет,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,cторонник


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [7]:
df.describe()

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


### Выгрузка датасета

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Дашборды и презентация по ссылке

https://public.tableau.com/views/Telecomm_17206407245830/1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link